# Mix em' Up!

Now, as we have see the power of `hoverData`, `clickData` and `selectedData` we will start leveraging their real power in this notebook. We will use them (ie. select or hover or click on the graph) and then the result accordingly will be reflected on another graphs! So basically a graph updates a graph!

Now, you will say "*Yea, now it looks kinds a Dashboard!*" <br>
—<br>
Let's get started!

# 

In [129]:
import dash
import dash_core_components as dcc, dash_html_components as html
from dash.dependencies import Output, Input

import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd, numpy as np
import json

### Loading the data

In [189]:
df = pd.read_csv("../data/data/mpg.csv")
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [190]:
df.horsepower = df.horsepower.apply(lambda x: float(x) if x != '?' else 0.0)

We will create 2 kinds of dashboards here for practice:
1. From the course
2. From a creative mind

###### 

## Simple One
*One Graph → Second Graph* (From course)

In [12]:
app = dash.Dash()

# Creating the initial graph for graph 1
plot = [
        go.Scatter(x=df.model_year + 1900,  # Added 1900 because the year
                   y=df.mpg,                # is in 78, 45... need 1978.
                   mode="markers",
                      )
]
layout = go.Layout(
            title="Some great stuff",
            hovermode="closest",
            xaxis=dict(title="← Model Year →"),
            yaxis=dict(title="← MPG →")
         )
figure = go.Figure(data=plot, layout=layout)
default_style={"width":"50%", "display":"inline-block"}

app.layout = html.Div(children=[
                html.Div([
                    dcc.Graph(id="graph-1", figure=figure)
                    ], style=default_style),
                html.Div([
                    dcc.Graph(id="graph-2")
                    ], style=default_style)
])

app.run_server()

If you run ↑ this. There will be the obvious output. But the problem is this ↓

<img src="problem.png" height=500 width=500>

Now, as they are stacked which is not a problem, but **to make it look nicer, we need to add jitter**. 

> But again, there is a problem. We don't have `jitter` parameter in the `go.Scatter` as we had in the `go.Box()`... so now?

In [22]:
# So we need to add jitter in the YEARS
years_jittered = df.model_year + np.random.randint(-5, 5, df.shape[0]) * 0.1

app = dash.Dash()
plot = [            ### POI ↓ POI ###
        go.Scatter(x=years_jittered + 1900, 
                   y=df.mpg,                
                   mode="markers",
            )
]
layout = go.Layout(
            title="Some great stuff",
            hovermode="closest",
            xaxis=dict(title="← Model Year →"),
            yaxis=dict(title="← MPG →")
         )
figure = go.Figure(data=plot, layout=layout)
default_style={"width":"50%", "display":"inline-block"}

app.layout = html.Div(children=[
                html.Div([
                    dcc.Graph(id="graph-1", figure=figure)
                    ], style=default_style),
                html.Div([
                    dcc.Graph(id="graph-2")
                    ], style=default_style)
])

app.run_server()

<img src="solution.png" height=500 width=500>

### Now we will correct the text displayed on hover
Because you see, as we have done the jitter in the years, the years are shown in 1977.3 and likewise. So we need to pass the real year in the as a `text` parameter.

In [30]:
app = dash.Dash()

years_jittered = df.model_year + np.random.randint(-5, 5, df.shape[0]) * 0.1
plot = [            
        go.Scatter(x=years_jittered + 1900, 
                   y=df.mpg,                
                   mode="markers",
                   text=df.model_year + 1900,  ### POI ← POI ### 
                   hoverinfo= 'text'           ### POI ← POI ###
            )
]
layout = go.Layout(
            title="Some great stuff",
            hovermode="closest",
            xaxis=dict(title="← Model Year →"),
            yaxis=dict(title="← MPG →")
         )
figure = go.Figure(data=plot, layout=layout)
default_style={"width":"50%", "display":"inline-block"}

app.layout = html.Div(children=[
                html.Div([
                    dcc.Graph(id="graph-1", figure=figure)
                    ], style=default_style),
                html.Div([
                    dcc.Graph(id="graph-2")
                    ], style=default_style)
])

app.run_server()

**Syntax Analysis**
```python
go.Scatter(...,  # ↓ This will be shown as extra
           text=df.model_year + 1900, 
           hoverinfo='text'
)         # ↑ This will show WHAT TO SHOW
```

<u>Q.</u> But **What if** you pass `hoverinfo='text'` but there is no 'text' defined?  <br>
<u>A.</u> Nothing will be shown on hover! Yeah! It is new.

# 

## Hoo... now we will create the *real* stuff
Till here, we just discovered how to show the data points in a nice way... but the main target of this book was to take an effect on the other graph. So, *it starts now actully*.

In [63]:
app = dash.Dash()
years_jittered = df.model_year + np.random.randint(-5, 5, df.shape[0]) * 0.1

plot = [
        go.Scatter(x=years_jittered + 1900, 
                   y=df.mpg,                
                   mode="markers",
                   text=df["name"]
            )
]
layout = go.Layout(
            title="Some great stuff",
            hovermode="closest",
            xaxis=dict(title="← Model Year →"),
            yaxis=dict(title="← MPG →")
         )
figure = go.Figure(data=plot, layout=layout)
default_style={"width":"50%", "display":"inline-block"}

app.layout = html.Div(children=[
                html.Div([
                    dcc.Graph(id="graph-1", figure=figure)
                    ], style=default_style),
                html.Div([
                    dcc.Graph(id="graph-2"),
                    html.Pre(id="preview")
                    ], style=default_style)
])


# Updates the label
@app.callback(Output("preview", "children"),
             [Input("graph-1", "hoverData")])
def update_graph2(hoverData):
                                        ### POI ↓ POI ###
    filtered = df.iloc[hoverData["points"][0]["pointIndex"]]
    
    # Extracting details to use later
    n_cylinders = filtered["cylinders"]
    displacement = filtered["displacement"]
    acceleration = filtered["acceleration"]
    name = filtered["name"].title()
    
    # Returning the stuff for <PRE>
    return f"""
    Name: {name}
    Number of cylinders: {n_cylinders}
    Displacement: {displacement}
    0 to 60 miles in: {acceleration} seconds
    """


@app.callback(Output("graph-2", "figure"),
             [Input("graph-1", "hoverData")])
def update_graph2(hoverData):
                                        ### POI ↓ POI ###
    filtered = df.iloc[hoverData["points"][0]["pointIndex"]]
    
    # Extracting stuff
    n_cylinders = filtered["cylinders"]
    name = filtered["name"].title()
    acceleration = filtered["acceleration"]
    
    # The plot is crazy
    plot = [go.Scatter(x=[0, 1], ### POI ← POI ###
                       y=[0, 60 / acceleration], ### POI ← POI ###
                       mode="lines", 
                       line=dict(width=int(n_cylinders)) ### POI ← POI ###
                )]   
                                       ### POI ↓ POI ↓ POI ↓ POI ↓ POI ###
    layout = go.Layout(title=name, margin=dict(l=0, r=0, t=50, b=0),
                       height=300, width=300, ### POI ← POI ###
                       xaxis=dict(visible=False), ### POI ← POI ###
                       yaxis=dict(visible=False, range=[0, 60 / df["acceleration"].min()])) ### POI ← POI ###
    figure = go.Figure(data=plot, layout=layout)
    return figure



app.run_server()

## Oh... wai wai wai... slowdown a bit!
I know I have thrown a lot of things together... but I think we are now capable of getting more stuff done at once. But still, let's do our: <br>
**Syntax Analysis**

### 1.
```python
filtered = df.iloc[hoverData["points"][0]["pointIndex"]]
```
Now this is an amazing thing. With `pointIndex` as the one of the parameters of that point, it will actully give the index of *THAT* row so that we won't have to do some error prone stuff like `df[df.col1 == "value"].iloc[0]` to grab the first row no matter how many returned. As it is error prone, we should use the above mentioned technique.

### 2.
```python
plot = [go.Scatter(x=[0, 1],  # | This will make the slope of the line
                              # ↓ instead of just x=[0, 1], y=[0, 1]
                   y=[0, 60 / acceleration], 
                   mode="lines", 
                   line=dict(width=int(n_cylinders))
            )]                   # ↑ Different thickness of line
```

### 3.
```python
layout = go.Layout(title=name, #↓ (1)
                   margin=dict(l=0, r=0, t=50, b=0),
                   height=300, width=300, # ← (2)
                   xaxis=dict(visible=False), 
                              # ↕ (3)
                   yaxis=dict(visible=False,
                              range=[0, 60 / df["acceleration"].min()]))
                              # ↑ (4)
```
(1) `LRTB` for Left, Right, Top, Bottom — Graph specific margins instead of CSS (*Note*: They are in the dict format in the `margin` parameter) <br>
(2) `Width and Height` are again the Graph Specific styles to manage the width and height of graph without using CSS <br>
(3) `visible` to toggle visibility of axes. <br>
(4) `range` Cool! This one again goes in the dict od axes and sets the range. Works like `plt.xlim([])`.

___

And there! We have successfully created an application / dashboard which uses the graph to update another graph!

# 

## Extra One
*Multiple Graphs → Multiple Reflection* (From Mind)

In [196]:
# Extracting Company name
df["company"] = df.name.str.partition(" ")[0].str.title()
df["model_year"] = df["model_year"] + 1900

Selectable Data:
1. Year
    - A range slider ✓✓
        - Filters data accordingly
2. Number of cylinders
    - A group of checkboxes ✓✓
        - Filters data accordingly

Graphs:
1. Mpg vs Weight Scatter
    - On HOVER: ✓✓
        - Show Card information ✓
            - Name
            - Released in
            - nth fastest car (in acceleration) from the total cars in that year
        - On HoverInfo ✓
            - N cylinders
            - horsepower
    - On CLICK: ✓✓
        - Horizontal Bar Chart  ✓
            - Of that car with the acceleration comparision with other<br>cars in the same year
        - Show Card information ✗
            - Name
            - Released in
            - nth fastest car (in acceleration) from the total cars in that year
    - On SELECT:
        - Show a density plot ✓
            - Density plot of horsepower of all cars in that selected area
        - Show bar chart 
            - Value counts for all data points by company name
            
2. Density plot for the selected 
3. Horizontal Bar Chart
4. Density plot
5. Bar Chart

—<br>**I think** this is much for now, let's do this!

In [197]:
df["text"] = "Cylinders: " + df.cylinders.astype(str) + " — Year: " + df.model_year.astype(str)

In [236]:
app = dash.Dash()

# Elements
# --------

# 1.
year_min = df.model_year.min()
year_max = df.model_year.max()
year_slider = dcc.RangeSlider(id="year-slider",
                              min=year_min,
                              max=year_max,
                              value=[year_min, year_max],
                              marks={i : i for i in range(year_min, year_max + 1, 2)})
                            
# 2.
cylinder_select = dcc.Checklist(id="cylinder-select",
                                options=[
                                    {'label': i, 'value': i}
                                    for i in df.cylinders.unique()
                                ],
                                values=df.cylinders.unique(),
                                labelStyle={'display': 'inline-block'})


# Layout
app.layout = html.Div(children=[ # MASTER DIV
                html.Div(children=[ # DIV for Year and Cylinder
                            html.Div([ # DIV for Year
                                html.Label("Year"),
                                year_slider
                            ]),
                            html.Div([ # DIV for Cylinder
                                html.Label("Number of Cylinders"),
                                cylinder_select    
                            ], style={"margin-top":"5%"})
                        ],
                         style=dict(
                            width="80%",
                            marginLeft="10%")
                ),
                html.Div(children=[ # DIV for Graph-1 (scatter)
                            dcc.Graph(id="main-scatter")
                        ],
                        style={"display":"inline-block"}),
                html.Div(children=[ # DIV for Label (card)
                            dcc.Markdown(id="card")
                        ],
                        style={"position":"absolute",
                               "margin-left":"60%", "margin-top":"-45%"}),
                html.Div(children=[ # DIV for Graph-2 (scatter)
                            dcc.Graph(id="h_bar")
                        ],
                        style={"position":"absolute",
                               "margin-left":"50%", "margin-top":"-27%"}),
                html.Div(children=[ # DIV for Graph-3 (dist)
                            dcc.Graph(id="distplot")
                        ],
                        style={"position":"absolute",
                               "margin-left":"5%", "margin-top":"3%"}),
                html.Div(children=[ # DIV for Graph-4 (bar)
                            dcc.Graph(id="v_bar")
                        ],
                        style={"position":"absolute",
                               "margin-left":"50%", "margin-top":"3%"}),
])

# Updating with Year and Cylinders
@app.callback(Output("main-scatter", "figure"),
             [Input("year-slider", "value"),
              Input("cylinder-select", "values")])
def filter_graph(year_range, cylinder_selected):
    """
    This function will update the graph based on year slider
    and the checkboxes for cylinders.
    """
    
    filter_ = (df.model_year >= year_range[0]) & (df.model_year <= year_range[1]) & \
                (df.cylinders.isin(cylinder_selected))
    filtered_df = df[filter_]
    
    plot = [go.Scatter(
                x=filtered_df.mpg,
                y=filtered_df.weight,
                mode="markers",
                marker=dict(opacity=0.5,
                            color="cyan",
                            size=10),
                text=filtered_df["text"])]
    
    syl_len = len(cylinder_selected)
    if syl_len == 5:
        cyl_show = "Cylinders: All"

    else:
        if syl_len == 1:
            cyl_show = "Cylinder: " + str(cylinder_selected[0])
        elif syl_len == 0:
            cyl_show = "Cylinder: Not Selected"
        else:
            cyl_show = ''
            for i in range(syl_len):
                if i == 0:
                    cyl_show += 'Cylinders: '
                elif i == syl_len - 1:
                    cyl_show += " and "
                else:
                    cyl_show += ", "
                cyl_show += str(cylinder_selected[i])
    
    layout = go.Layout(title=f"Year Range {year_range[0]} — {year_range[1]} | {cyl_show}",
                   xaxis=dict(title="← MPG →"),
                   yaxis=dict(title="← Weight →"),
                   width=900, height=670,
                   margin=dict(l=1))
    return go.Figure(data=plot, layout=layout)
    
    
    
# Working with hover
@app.callback(Output("card", "children"),
             [Input("main-scatter", "hoverData")])
def hover_work(hoverData):
    """
    This will just show the information in a markdown 
    dcc widget with the rank of car.
    """
    
    iloc = hoverData["points"][0]["pointIndex"]
    filtered_df = df.iloc[iloc]
    name = filtered_df["name"]
    year = filtered_df["model_year"]
    acceleration = filtered_df["acceleration"]
    
    filtered_df = df[df.model_year == year]
    th_rank = filtered_df['acceleration'].rank(ascending=False)[iloc]
    outof = filtered_df.shape[0]
    
    markdown = f"""
# {name.title()}
has scored  
### {int(th_rank)}th  
rank  
outof  
### {outof} cars in {year} year.
    """
    return markdown


# Working with Click
@app.callback(Output("h_bar", "figure"),
             [Input("main-scatter", "clickData")])
def handle_click(clickData):
    iloc = clickData["points"][0]["pointIndex"]
    that_year = df.iloc[iloc]["model_year"]
    filtered_df = df[df.model_year == that_year].sort_values(by="acceleration")
    filtered_df.drop_duplicates(inplace=True, subset=["name"])
    filtered_df["name"] = filtered_df["name"].str.title()
    
    df_single = filtered_df[filtered_df.index == iloc]
    filtered_df.loc[iloc, "acceleration"] = 0
    
    plot = [go.Bar(x=filtered_df.acceleration,
                   y=filtered_df["name"],
                   orientation="h",
                   hoverinfo='x',
                   name="Other Cars"
                ),
            go.Bar(x=df_single.acceleration,
                   y=df_single["name"],
                   orientation="h",
                   marker=dict(color="gold"),
                   offset=-0.4,
                   hoverinfo='x',
                   name=df_single["name"].values[0]
                )
           ]
    layout = go.Layout(title=f"Car's acceleration in {that_year}",
                       margin=dict(l=250),
                       xaxis=dict(title="← Acceleration →"))
    fig = go.Figure(data=plot, layout=layout)
    return fig


# Working with selection
@app.callback(Output("distplot", "figure"),
             [Input("main-scatter", "selectedData")])
def handle_select_1(selectedData):
    n_points = len(selectedData["points"])
    selected_ilocs = []
    
    for i in range(n_points):
        selected_ilocs.append(selectedData["points"][i]["pointIndex"])
        
    filtered_df = df.iloc[selected_ilocs]
    fig = ff.create_distplot([filtered_df.horsepower.values.tolist()], 
                             group_labels=["Horsepower"],
                             show_hist=False, show_rug=False)
    fig["layout"]["title"] = f"Horsepower distribution for {n_points} selected car(s)"
    return fig

@app.callback(Output("v_bar", "figure"),
             [Input("main-scatter", "selectedData")])
def handle_select_2(selectedData):
    n_points = len(selectedData["points"])
    selected_ilocs = []
    
    for i in range(n_points):
        selected_ilocs.append(selectedData["points"][i]["pointIndex"])
        
    filtered_df = df.iloc[selected_ilocs]
    counts = filtered_df.company.value_counts()
    plot = [go.Bar(x=counts.index,
                   y=counts.values,
                   marker=dict(color=counts.values))]
    layout = go.Layout(title="Company frequency for selected cars",
                       xaxis=dict(title="Company"),
                       yaxis=dict(title="← Frequency (count) →"))
    figure = go.Figure(data=plot, layout=layout)
    return figure

    
app.run_server()

# Done!

Here is how it looks ↓
<img src="done.png">
<img src="done2.png">

# 

I know... it doesn't look masterpiece. That is because I haven't done any kind of theme applying or so... but this is the first one!

Please review the code as line by line. I have not given the comment in those area but still it is pretty readable.

# 

# Next up,
A new section with more fun is coming in the way!